# Database Connection

In [1]:
from pymongo import MongoClient

#MongoDB Connection
client = MongoClient('mongodb://localhost:27017/')

# Create Database and Collection

In [2]:
# Create Database
db = client['VideoKeyframeDB']

# Create the Collection
collection = db['videos']

# Prepare and insert Data

In [3]:
import os
import cv2
import gridfs

fs = gridfs.GridFS(db)

def parse_shot_boundaries(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()
        shots = [line.strip().split(', ') for line in lines]
        return shots

def extract_framerate(video_path):
    video = cv2.VideoCapture(video_path)
    framerate = video.get(cv2.CAP_PROP_FPS)
    video.release()
    return framerate

def read_keyframe_image(file_path):
    with open(file_path, 'rb') as f:
        return f.read()

def prepare_video_data(video_id, collection_name, video_path, shot_boundaries_path, keyframes_path):
    original_framerate = extract_framerate(video_path)
    shots = parse_shot_boundaries(shot_boundaries_path)
    frames = []
    for i, (start, end) in enumerate(shots, start=1):
        frame_id = f"{video_id}_Scene-{i}"
        keyframe_path = os.path.join(keyframes_path, f"{frame_id}_.jpg")
        print(keyframe_path)
        if os.path.exists(keyframe_path):
            keyframe_image = read_keyframe_image(keyframe_path)
            keyframe_file_id = fs.put(keyframe_image, filename=f"{frame_id}.jpg")
            frame = {
                "FrameID": frame_id,
                "Starttime": start,
                "Endtime": end,
                "KeyframeImageID": keyframe_file_id,
                "Objects": []
            }
            frames.append(frame)
        else: 
            print(f"Fehlendes Keyframe Bild: {keyframe_path}")

    video_data = {
        "VideoID": video_id,
        "CollectionName": collection_name,
        "OriginalFramerate": original_framerate,
        "EditedFramerate": 24,
        "Frames": frames
    }
    return video_data

def insert_all_videos(base_video_path, base_shot_boundaries_path, base_keyframes_path, collection_name):
    for video_num in range(100, 200): # from 00100 to 00199
        video_id = f"{video_num:05d}"
        video_path = os.path.join(base_video_path, video_id, f"{video_id}.mp4")
        shot_boundaries_path = os.path.join(base_shot_boundaries_path, f"{video_id}_shots.txt")
        keyframes_path = os.path.join(base_keyframes_path, video_id)

        if os.path.exists(shot_boundaries_path) and os.path.exists(keyframes_path):
            video_data = prepare_video_data(video_id, collection_name, video_path, shot_boundaries_path, keyframes_path)
            collection.insert_one(video_data)
        else:
            print(f"Missing Video Data for Video {video_id}: {shot_boundaries_path} or {keyframes_path}")

#Change according to config
base_video_path = 'V3C1-100'
base_shot_boundaries_path = 'shot_boundaries_converted'
base_keyframes_path = 'keyframes'
collection_name = 'V3C100'

insert_all_videos(base_video_path, base_shot_boundaries_path, base_keyframes_path, collection_name)

ModuleNotFoundError: No module named 'cv2'

# Update Framrates (Do not execute if originalFramrate and Edited Framrate are already set initally)

In [7]:
import os
import cv2
import pymongo

# Verbinde zur MongoDB
client = pymongo.MongoClient('mongodb://localhost:27017/')
db = client['VideoKeyframeDB']
collection = db['videos']

def extract_framerate(video_path):
    video = cv2.VideoCapture(video_path)
    framerate = video.get(cv2.CAP_PROP_FPS)
    video.release()
    return framerate

def update_video_framerates(base_video_path):
    videos = collection.find({})
    for video in videos:
        video_id = video['VideoID']
        video_path = os.path.join(base_video_path, video_id, f"{video_id}.mp4")

        if os.path.exists(video_path):
            original_framerate = extract_framerate(video_path)
            collection.update_one(
                {"VideoID": video_id},
                {"$set": {
                    "OriginalFramerate": original_framerate,
                    "EditedFramerate": 24
                }}
            )
            print(f"Updated VideoID: {video_id} with OriginalFramerate: {original_framerate} and EditedFramerate: 24")
        else:
            print(f"Video file not found: {video_path}")

# Change according to your config
base_video_path = 'V3C1-100'

update_video_framerates(base_video_path)

Updated VideoID: 00100 with OriginalFramerate: 23.976 and EditedFramerate: 24
Updated VideoID: 00101 with OriginalFramerate: 29.97002997002997 and EditedFramerate: 24
Updated VideoID: 00102 with OriginalFramerate: 29.97002997002997 and EditedFramerate: 24
Updated VideoID: 00103 with OriginalFramerate: 29.97002997002997 and EditedFramerate: 24
Updated VideoID: 00104 with OriginalFramerate: 29.97002997002997 and EditedFramerate: 24
Updated VideoID: 00105 with OriginalFramerate: 25.0 and EditedFramerate: 24
Updated VideoID: 00106 with OriginalFramerate: 25.0 and EditedFramerate: 24
Updated VideoID: 00107 with OriginalFramerate: 29.97002997002997 and EditedFramerate: 24
Updated VideoID: 00108 with OriginalFramerate: 29.97002997002997 and EditedFramerate: 24
Updated VideoID: 00109 with OriginalFramerate: 25.0 and EditedFramerate: 24
Updated VideoID: 00110 with OriginalFramerate: 25.0 and EditedFramerate: 24
Updated VideoID: 00111 with OriginalFramerate: 30.0 and EditedFramerate: 24
Updated 

# Update Database for correct converted shot_boundaries (Do not execute if you initially already have used converted shot_boundaries)

In [5]:
from pymongo import MongoClient
import os

# Configuration
client = pymongo.MongoClient('mongodb://localhost:27017/')
db = client['VideoKeyframeDB']
collection = db['videos']

def parse_shot_boundaries(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()
        shots = [line.strip().split(', ') for line in lines]
        return shots

def update_shot_boundaries(video_id, converted_shot_boundaries_file):
    shots = parse_shot_boundaries(converted_shot_boundaries_file)
    for i, (start, end) in enumerate(shots, start=1):
        frame_id = f"{video_id}_Scene-{i}"
        collection.update_one(
            {"VideoID": video_id, "Frames.FrameID": frame_id},
            {"$set": {
                "Frames.$.Starttime": start,
                "Frames.$.Endtime": end
            }}
        )
    print(f"Updated shot boundaries for video {video_id}")

def update_all_videos(converted_shot_boundaries_path):
    for video_num in range(100, 200):  # from 00100 to 00199
        video_id = f"{video_num:05d}"
        converted_shot_boundaries_file = os.path.join(converted_shot_boundaries_path, f"{video_id}_shots.txt")
        
        if os.path.exists(converted_shot_boundaries_file):
            print(f"Updating shot boundaries for video: {video_id}")
            update_shot_boundaries(video_id, converted_shot_boundaries_file)
        else:
            print(f"Missing converted shot boundaries file: {converted_shot_boundaries_file}")

if __name__ == "__main__":
    update_all_videos(converted_shot_boundaries_path)


Updating shot boundaries for video: 00100
Updated shot boundaries for video 00100
Updating shot boundaries for video: 00101
Updated shot boundaries for video 00101
Updating shot boundaries for video: 00102
Updated shot boundaries for video 00102
Updating shot boundaries for video: 00103
Updated shot boundaries for video 00103
Updating shot boundaries for video: 00104
Updated shot boundaries for video 00104
Updating shot boundaries for video: 00105
Updated shot boundaries for video 00105
Updating shot boundaries for video: 00106
Updated shot boundaries for video 00106
Updating shot boundaries for video: 00107
Updated shot boundaries for video 00107
Updating shot boundaries for video: 00108
Updated shot boundaries for video 00108
Updating shot boundaries for video: 00109
Updated shot boundaries for video 00109
Updating shot boundaries for video: 00110
Updated shot boundaries for video 00110
Updating shot boundaries for video: 00111
Updated shot boundaries for video 00111
Updating shot bo